# Ecuaciones de Euler

Las ecuaciones de Navier-Stokes compresibles se pueden escribir de la siguiente forma

$$\frac{\partial \rho}{\partial t} + \nabla (\rho \underline{u}) = 0$$

$$\frac{\partial \rho \underline{u}}{\partial t} + \nabla (\rho \underline{uu})+\nabla P = \rho \underline{F}$$

Donde $\underline{u}$ es el vector de velocidad, $\rho$ la densidad, $\nu$ la viscosidad dinámica, $\underline{F}$ la fuerzas externas por unidad de volumen y $\underline{uu}$ el tensor $(\underline{uu})_{ij}=u_i u_j$

La primera de estas ecuaciones es la conservación de la masa y la segunda es la conservación de momento. Para el caso de tratar con liquidos se puede hacer la aproximación de fluido incompesible, es decir la densidad es constante, $\rho = \text{cte}$.

Bajo esta aproximación se las ecuaciones de Navier-Stokes se reducen a las siguientes

$$\nabla \underline{u} = 0$$

$$\frac{\partial \underline{u}}{\partial t} + (\underline{u}|\cdot\nabla) \underline{u}+\frac{1}{\rho}\nabla P = \underline{F}$$

Resulta que tomando la divergencia de la ecuación de momento se puede mostrar que se obtiene una ecuación de Poisson para la presión cuando se considera el tiempo como continuo.

$$\frac{\partial^2 P}{\partial x^2}+\frac{\partial^2 P}{\partial y^2} = \rho\left(\frac{\partial F}{\partial x}+\frac{\partial G}{\partial y}\right)-\rho \left(\left(\frac{\partial u}{\partial x}\right)^2+2\frac{\partial u}{\partial y}\frac{\partial v}{\partial x}+\left(\frac{\partial v}{\partial y}\right)^2 \right)$$

Para el caso discontinuo se llega a otra ecuación que se debe de resolver otra ecuación

$$\frac{\partial^2 P}{\partial x^2}+\frac{\partial^2 P}{\partial y^2} = \rho\left(\frac{\partial F}{\partial x}+\frac{\partial G}{\partial y}\right)-\rho \left(\frac{\partial}{\partial t}\left(\frac{\partial u}{\partial x}+\frac{\partial v}{\partial y}\right)+\left(\frac{\partial u}{\partial x}\right)^2+2\frac{\partial u}{\partial y}\frac{\partial v}{\partial x}+\left(\frac{\partial v}{\partial y}\right)^2 \right)$$

Notemos que en el caso discontinuo aparece el término $$\frac{\partial}{\partial t}\left(\frac{\partial u}{\partial x}+\frac{\partial v}{\partial y}\right)$$
Que debería ser cero por la ecuación de continuidad, sin embargo debido a errores numéricos en general $$\frac{\partial u}{\partial x}+\frac{\partial v}{\partial y}\neq 0$$ por lo tanto se debe de tomar en cuenta como término de corrección.

Donde $\underline{u} = (u,v)$ y $\underline{F} = (F,G)$ y 

Por lo tanto llegamos a un sistema de ecuaciones diferenciales parciales para nuestras tres variables

$$
\frac{\partial u}{\partial t} + u\frac{\partial u}{\partial x} + v\frac{\partial u}{\partial x} = -\frac{1}{\rho}\frac{\partial P}{\partial x}+F\\
\frac{\partial v}{\partial t} + u\frac{\partial v}{\partial x} + v\frac{\partial v}{\partial x} = -\frac{1}{\rho}\frac{\partial P}{\partial y}+G\\
\frac{\partial^2 P}{\partial x^2}+\frac{\partial^2 P}{\partial y^2} =\rho\left(\frac{\partial F}{\partial x}+\frac{\partial G}{\partial y}\right)-\rho \left(\left(\frac{\partial u}{\partial x}\right)^2+2\frac{\partial u}{\partial y}\frac{\partial v}{\partial x}+\left(\frac{\partial v}{\partial y}\right)^2 \right)$$


Otra forma de ver el sistema de ecuaciones anterior es de la siguiente forma

$$\frac{\partial \bf{U}}{\partial t} + \frac{\partial \bf{F}}{\partial x} + \frac{\partial \bf{G}}{\partial y} = \bf{f}$$

Donde 
$$ 
\bf{U} = \it{(u, v)}\\
\bf{F} = \it{(u^2+\frac{P}{\rho}, uv)}\\
\bf{G} = \it{(uv, v^2+\frac{P}{\rho})}\\
\bf{f} = \it{(F, G)}
$$

In [1]:
#Se importan las librerias necesarias
from numpy import *
from matplotlib.pyplot import *
from matplotlib import animation
from mpl_toolkits.mplot3d import Axes3D
%matplotlib inline
# importar el script para redireccionar el video 
%run 'libreria_video_html.ipynb'

## Lax - Friedrics

In [2]:
def presion_poisson(u, v, p, dx, dy, dt, CondFrontP, N_iteraciones):
    f = zeros_like(u)
    pn = zeros_like(v)
    
    f[1:-1,1:-1] = (u[1:-1,2:]-u[1:-1,0:-2])/(dx*dt) + (v[2:,1:-1]-v[0:-2,1:-1])/(dy*dt)\
                    -(0.5*(u[1:-1,2:]-u[1:-1,0:-2])/dx)**2-(0.5*(v[2:,1:-1]-v[0:-2,1:-1])/dy)**2\
                    -(0.5*(v[1:-1,2:]-v[1:-1,0:-2])/dx)*(0.5*(u[2:,1:-1]-u[0:-2,1:-1])/dy)
    pn = p.copy()
    for n in range(N_iteraciones):
        p[1:-1,1:-1] = (dy**2*(pn[1:-1,2:]+pn[1:-1,0:-2,])+dx**2*(pn[2:,1:-1]+pn[0:-2,1:-1])-\
                        dx**2 *dy**2 *f[1:-1,1:-1])/(2*(dx**2+dy**2))
        
        pn = CondFrontP(u, v, p, dx, dy, dt)
        
    return pn

In [3]:
def U_vec(u,v):
    uu = []
    uu.append(u)
    uu.append(v)
    return uu

def F_vec(u, v, p):
    F1 = zeros_like(u)
    F2 = zeros_like(u)
    F1[1:-1,1:-1] = u[1:-1,1:-1]**2+p[1:-1,1:-1]
    F2[1:-1,1:-1] = u[1:-1,1:-1]*v[1:-1,1:-1]
    f = [F1,F2]
    return f

def G_vec(u, v, p):
    G1 = zeros_like(u)
    G2 = zeros_like(u)
    G1[1:-1,1:-1] = u[1:-1,1:-1]*v[1:-1,1:-1]
    G2[1:-1,1:-1] = v[1:-1,1:-1]**2+p[1:-1,1:-1]
    g = [G1,G2]
    return g

In [4]:
def discret_velLaxFried(u, v, p, dx, dy, dt, CondFrontU):
    U = U_vec(u, v)
    F = F_vec(u, v, p)
    G = G_vec(u, v, p)
    Un = [[],[]]
    Un[0] = U[0].copy()
    Un[1] = U[1].copy()
    for i in [0,1]:
        U[i][1:-1,1:-1] = (Un[i][2:,1:-1]+Un[i][0:-2,1:-1]+Un[i][1:-1,2:]+Un[i][1:-1,0:-2])/4.\
                -(0.5*dt/dx)*(F[i][1:-1,2:]-F[i][1:-1,0:-2])\
                -(0.5*dt/dy)*(G[i][2:,1:-1]-G[i][0:-2,1:-1])
                
    Un = CondFrontU(U, p, dx, dy, dt)
    return Un[0], Un[1]

In [5]:
def avance_en_tiempo(u0, v0, p0, dx, dy, nx, ny, nt, sigma, C, N_iteraciones, CondFrontP, CondFrontU, 
                           esquema_vel, flag = False):
    um = zeros((ny, nx))
    vm = zeros((ny, nx))
    pm = zeros((ny, nx))
    
    u = u0.copy()
    v = v0.copy()
    p = p0.copy()
    
    if flag:
        Uh = []
        Vh = []
        Ph = []
    t = 0.
    
    for n in range(nt):
        umax = amax(absolute(u))
        vmax = amax(absolute(v))
        
        if umax <> 0. :
            if vmax <> 0. : dt = C*sigma*min(dx/umax, dy/vmax, 1.0e-4)
            else : dt = C*sigma*min(dx/umax, 1.0e-4)
        else : dt = 1.0e-4
        
        t = t + dt
        
        um = u.copy()
        vm = v.copy()
        pm = p.copy()
        
        if flag:
            Uh.append(um)
            Vh.append(vm)
            Ph.append(pm)

        p =  presion_poisson(um, vm, pm, dx, dy, dt, CondFrontP, N_iteraciones)
        u, v = esquema_vel(um, vm, p, dx, dy, dt, CondFrontU)
        
    if flag:
        return Uh, Vh, Ph
    else:
        return u, v, p

In [6]:
lx = 1.
ly = 1.
nx = 91
ny = 91
nt = 300
sigma = 0.5
N_int = 1000
dx = lx/(nx-1)
dy = ly/(ny-1)

u0 = zeros((ny,nx))
v0 = zeros((ny,nx))
p0 = zeros((ny,nx))

U0 = [u0, v0]
N = 4

def CondFrontUU(U, p, dx, dy, dt):
    U[0][1:-1,0] = 1.     #u en x=0
    U[0][1:-1,-1] = 1.     #u en x=lx
    U[0][0,1:-1] = 0.     #u en y=0
    U[0][-1,1:-1] = 0.     #u en y=ly
    U[1][1:-1,0] = 0.     #v en x=0
    U[1][1:-1,-1] = 0.     #v en x=lx
    U[1][0,1:-1] = 0.     #v en y=0
    U[1][-1,1:-1] = 0.     #v en y=ly
    return U

def CondFrontPP(u, v, p, dx, dy, dt):
    p[1:-1,0] = 0.
    p[1:-1,-1] = 0.
    p[0,1:-1] = 0.
    p[-1,1:-1] = 0.
    return p

Uh, Vh, Ph = avance_en_tiempo(u0, v0, p0, dx, dy, nx, ny, nt, sigma, 1., N_int, 
                              CondFrontPP, CondFrontUU, discret_velLaxFried, True)

UMag = [sqrt(Uh[i]**2+Vh[i]**2) for i in range(0, nt)]

x = linspace(0., lx, nx)
y = linspace(0., ly, ny)
X, Y = meshgrid(x, y)

# Se define el ambiente en el que queremos hacer la animación
figanim = figure(figsize=(8,7))
Ax=figanim.add_subplot(111)
Ax.set_xlim(0,lx)
Ax.set_ylim(0,ly)
cbar_ax = figanim.add_axes([0.85, 0.15, 0.05, 0.7])

def init():
    Ax.cla()
    campo_vectorial = Ax.quiver([],[],[],[])
    return campo_vectorial,

# Esta funcion se llama de manera secuencial para cada elemento i.
def animate(i,Ax,fig):
    NN = i
    Ax.cla()
    cbar_ax.cla()
    figanim.subplots_adjust(right=0.8)
    contorno = Ax.contour(X, Y, UMag[NN], 25)
    contornof = Ax.contourf(X, Y, UMag[NN], 25)
    figanim.colorbar(contornof, cbar_ax)
    campo_vectorial = Ax.quiver(X[::N,::N], Y[::N,::N], Uh[NN][::N,::N], Vh[NN][::N,::N])
    return campo_vectorial, contorno, contornof,

# Se llama a la animacion.  blit=True es para que solo se dibije las partes de la imagen que tienen cambios.
animation.FuncAnimation(figanim, animate, init_func=init, frames=nt, fargs=(Ax, figanim), interval=5)
#anim.save("Animacion.mp4", fps=10)

## Lax - Wendroff

In [ ]:
def discret_velLaxWen(u, v, p, dx, dy, dt, CondFrontU):
    U = U_vec(u, v)
    F = F_vec(u, v, p)
    G = G_vec(u, v, p)
    Un =  [[],[]]
    U05me =  [[],[]]
    U05ma =  [[],[]]
    
    Un[0] = U[0].copy()
    Un[1] = U[1].copy()
    U05ma[0] = U[0].copy()
    U05ma[1] = U[1].copy()
    U05me[0] = U[0].copy()
    U05me[1] = U[1].copy()
    
    for i in range(1):
        U05ma[i][1:-1,1:-1] = (Un[i][2:,1:-1]+Un[i][2:,2:]+Un[i][2:,0:-2]+Un[i][1:-1,1:-1])/4.\
                    -(0.5*dt/dx)*(F[i][1:-1,2:]-F[i][1:-1,1:-1])\
                    -(0.5*dt/dy)*(G[i][2:,2:]-G[i][0:-2,2:])
                
        U05me[i][1:-1,1:-1] = (Un[i][0:-2,1:-1]+Un[i][0:-2,2:]+Un[i][0:-2,0:-2]+Un[i][1:-1,1:-1])/4.\
                    -(0.5*dt/dx)*(F[i][1:-1,1:-1]-F[i][1:-1,0:-2])\
                    -(0.5*dt/dy)*(G[i][2:,0:-2]-G[i][0:-2,0:-2])

    F05ma = F_vec(U05ma[0], U05ma[1], p)
    F05me = F_vec(U05me[0], U05me[1], p)
    G05ma = G_vec(U05ma[0], U05ma[1], p)
    G05me = G_vec(U05me[0], U05me[1], p)

    for i in range(1):
        U[i][1:-1,1:-1] = Un[i][1:-1,1:-1]-(0.5*dt/dx)*(F05ma[i][1:-1,2:]-F05me[i][1:-1,0:-2])\
                            -(0.5*dt/dy)*(G05ma[i][2:,1:-1]-G05me[i][0:-2,1:-1])
    
    Un = CondFrontU(U, p, dx, dy, dt)

    return Un[0], Un[1]

In [ ]:
lx = 1.
ly = 1.
nx = 101
ny = 101
nt = 1000
sigma = 0.5
N_int = 1500
dx = lx/(nx-1)
dy = ly/(ny-1)

u0 = zeros((ny,nx))
v0 = zeros((ny,nx))
p0 = zeros((ny,nx))

U0 = [u0, v0]
N = 4

def CondFrontUU(U, p, dx, dy, dt):
    U[0][1:-1,0] = 1.     #u en x=0
    U[0][1:-1,-1] = 1.     #u en x=lx
    U[0][0,1:-1] = 0.     #u en y=0
    U[0][-1,1:-1] = 0.     #u en y=ly
    U[1][1:-1,0] = 0.     #v en x=0
    U[1][1:-1,-1] = 0.     #v en x=lx
    U[1][0,1:-1] = 0.     #v en y=0
    U[1][-1,1:-1] = 0.     #v en y=ly
    return U

def CondFrontPP(u, v, p, dx, dy, dt):
    p[1:-1,0] = 0.
    p[1:-1,-1] = 0.
    p[0,1:-1] = 0.
    p[-1,1:-1] = 0.
    return p

Uh, Vh, Ph = avance_en_tiempo(u0, v0, p0, dx, dy, nx, ny, nt, sigma, 1./sqrt(2.), N_int, 
                              CondFrontPP, CondFrontUU, discret_velLaxWen, True)

UMag = [sqrt(Uh[i]**2+Vh[i]**2) for i in range(0, nt)]

x = linspace(0., lx, nx)
y = linspace(0., ly, ny)
X, Y = meshgrid(x, y)

# Se define el ambiente en el que queremos hacer la animación
figanim = figure(figsize=(8,7))
Ax=figanim.add_subplot(111)
Ax.set_xlim(0,lx)
Ax.set_ylim(0,ly)
cbar_ax = figanim.add_axes([0.85, 0.15, 0.05, 0.7])

def init():
    Ax.cla()
    campo_vectorial = Ax.quiver([],[],[],[])
    return campo_vectorial,

# Esta funcion se llama de manera secuencial para cada elemento i.
def animate(i,Ax,fig):
    NN = i
    Ax.cla()
    cbar_ax.cla()
    figanim.subplots_adjust(right=0.8)
    contorno = Ax.contour(X, Y, UMag[NN], 25)
    contornof = Ax.contourf(X, Y, UMag[NN], 25)
    figanim.colorbar(contornof, cbar_ax)
    campo_vectorial = Ax.quiver(X[::N,::N], Y[::N,::N], Uh[NN][::N,::N], Vh[NN][::N,::N])
    return campo_vectorial, contorno, contornof,

# Se llama a la animacion.  blit=True es para que solo se dibije las partes de la imagen que tienen cambios.
animation.FuncAnimation(figanim, animate, init_func=init, frames=nt, fargs=(Ax, figanim), interval=5)
#anim.save("Animacion.mp4", fps=10)